<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-the-file" data-toc-modified-id="Load-the-file-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load the file</a></span></li><li><span><a href="#Drop-missing-rows" data-toc-modified-id="Drop-missing-rows-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Drop missing rows</a></span></li><li><span><a href="#Clean-activities" data-toc-modified-id="Clean-activities-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Clean activities</a></span><ul class="toc-item"><li><span><a href="#Regex-approach" data-toc-modified-id="Regex-approach-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Regex approach</a></span></li><li><span><a href="#Same-but-function" data-toc-modified-id="Same-but-function-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Same but function</a></span></li></ul></li></ul></div>

# Load the file

In [1]:
import pandas as pd

In [6]:
df=pd.read_csv('/home/mauritian/Downloads/attacks.csv', encoding='latin-1')

In [8]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


# Drop missing rows

In [24]:
df=df.drop(df[df.isna().sum(axis=1)>df.shape[1]-3].index).copy()

# Clean activities

In [26]:
df.Activity.nunique()

1532

In [27]:
df.Activity.describe()

count        5758
unique       1532
top       Surfing
freq          971
Name: Activity, dtype: object

In [28]:
df.Activity.unique()

array(['Paddling', 'Standing', 'Surfing', ...,
       'Crew swimming alongside their anchored ship',
       '4 men were bathing', 'Wreck of  large double sailing canoe'],
      dtype=object)

In [37]:
bag_of_words=[j for i in df.Activity.str.lower().dropna().values for j in i.split()]

In [39]:
print(len(bag_of_words),'words')
print(len(set(bag_of_words)), 'unique words')

14559 words
2240 unique words


## Regex approach

In [40]:
import re

In [45]:
re.findall(r'[a-z]+','boogie & dancing')

['boogie', 'dancing']

In [44]:
'boogie & dancing'.split()

['boogie', '&', 'dancing']

In [52]:
bag_of_words=[j for i in df.Activity.str.lower().dropna().values for j in re.findall(r'[a-z_]{3,}',i)]

In [53]:
print(len(bag_of_words),'words')
print(len(set(bag_of_words)), 'unique words')

12796 words
1782 unique words


In [66]:
words=list(map(lambda x: re.fullmatch(r'(.+?)(?:ing)?(?:ed)?(?:er)?\b',x).group(1),bag_of_words))

In [67]:
len(words)

12796

In [162]:
categories=set(pd.Series(words).value_counts().head(30).index)

In [163]:
categories=categories-set(['aft','for','free','from','his','into','sea','shark','swimm','the','was','wat','with','body']) | set(['swim','paddl', 'walk','feed','canoe','raft','float','sail'])

In [164]:
len(categories)

24

In [165]:
df.Activity.fillna('missing value', inplace=True)

In [166]:
for i in categories:
    df.loc[df.Activity.str.lower().str.contains(i), 'New_Activity']=i

In [167]:
categories

{'bath',
 'board',
 'boat',
 'canoe',
 'capsiz',
 'div',
 'feed',
 'fell',
 'fish',
 'float',
 'kayak',
 'overboard',
 'paddl',
 'raft',
 'sail',
 'scuba',
 'ship',
 'snorkel',
 'spearfish',
 'stand',
 'surf',
 'swim',
 'wad',
 'walk'}

In [168]:
df[['Activity','New_Activity']]

,Activity,New_Activity
0,Paddling,paddl
1,Standing,stand
2,Surfing,surf
3,Surfing,surf
4,Free diving,div
...,...,...
6297,Diving,div
6298,Pearl diving,div
6299,Swimming,swim
6300,missing value,missing value


In [169]:
df.New_Activity.value_counts()

surf             1163
fish             1090
swim             1073
missing value     544
other             497
div               461
wad               189
bath              186
stand             152
scuba             145
board             142
ship               91
snorkel            90
overboard          77
kayak              61
boat               59
float              57
paddl              36
raft               35
capsiz             33
walk               29
canoe              29
sail               23
fell               21
feed               18
body                1
Name: New_Activity, dtype: int64

In [170]:
df.New_Activity.isna().sum()

0

In [171]:
df.loc[df.Activity=='missing value','New_Activity']='missing value'

In [172]:
df.New_Activity.fillna('other',inplace=True)

In [173]:
df.New_Activity.value_counts()

surf             1163
fish             1090
swim             1073
missing value     544
other             497
div               461
wad               189
bath              186
stand             152
scuba             145
board             142
ship               91
snorkel            90
overboard          77
kayak              61
boat               59
float              57
paddl              36
raft               35
capsiz             33
walk               29
canoe              29
sail               23
fell               21
feed               18
body                1
Name: New_Activity, dtype: int64

## Same but function

In [180]:
def string_categorize(df, col, required_categories=[], useless_categories=[]):
    
    #create a copy of df
    df=df.copy()
    
    print(f'The number of unique values in the column {col} is {df[col].nunique()}')
    display(df[col].describe())
    
    bag_of_words=[j for i in df[col].str.lower().dropna().values for j in re.findall(r'[a-z_]+',i)]
    
    print(len(bag_of_words),'words')
    print(len(set(bag_of_words)), 'unique words')
    
    
    
    categories=set(pd.Series(bag_of_words).value_counts().head(30).index)
    
    categories=categories-set(useless_categories) | set(required_categories)
    
    print(f'Number of categories selected is {len(categories)}')
    
    df[col].fillna('missing value', inplace=True)
    
    for i in categories:
        df.loc[df[col].str.lower().str.contains(i), f'New_{col}']=i
        
    display(df[f'New_{col}'].value_counts())
    
    
    df.loc[df[col]=='missing value',f'New_{col}']='missing value'
    
    df[f'New_{col}'].fillna('other',inplace=True)    
    
    return df

In [181]:
string_categorize(df, 'Species ')

The number of unique values in the column Species  is 1549


count            3464
unique           1549
top       White shark
freq              163
Name: Species , dtype: object

11973 words
547 unique words
Number of categories selected is 30


a               2850
shark           2701
death            195
invalid          103
blacktip         101
nurse             92
questionable      84
bronze            67
by                62
blue               7
or                 5
possibly           3
whaler             3
m                  2
was                1
to                 1
Name: New_Species , dtype: int64

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,New_Activity,New_Species
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN,paddl,shark
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN,stand,missing value
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN,surf,missing value
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN,surf,shark
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN,div,shark
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN,div,missing value
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN,div,missing value
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN,swim,missing value
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",missing value,Jules Patterson,M,...,ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN,missing value,missing value


In [182]:
string_categorize(df, 'Sex ')

The number of unique values in the column Sex  is 6


count     5737
unique       6
top          M
freq      5094
Name: Sex , dtype: object

5736 words
4 unique words
Number of categories selected is 4


m      5661
f       637
n         2
lli       1
Name: New_Sex , dtype: int64

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,New_Activity,New_Sex
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN,paddl,f
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN,stand,f
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN,surf,m
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN,surf,m
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN,div,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN,div,m
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN,div,m
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN,swim,m
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",missing value,Jules Patterson,M,...,ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN,missing value,m
